Написать программу, которая шлёт в некоторый канал в Телеграмме последнюю новость с сайта https://vc.ru
Условие: в коде не должно быть токенов.


In [50]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [51]:
from requests.models import HTTPError


def parse_site():
    r = requests.get('https://vc.ru/new')
    if(r.status_code != 200):
        raise HTTPError()

    soup = BeautifulSoup(r.text)
    news = soup.find_all('a', {'class':"content-header__item content-header-number"})
    res = []
    for n in news:
        link = n.get('href')
        #убираем таиймзому из времени
        timeStr = n.find('time').get('title').split()
        time = datetime.strptime(timeStr[0] + timeStr[1], '%d.%m.%Y%H:%M:%S')
        res.append([time, link])

    res.sort(key=lambda x:x[0], reverse=True)
    return res[0]

In [52]:
r = parse_site()

In [53]:
r

[datetime.datetime(2022, 3, 27, 16, 43, 5),
 'https://vc.ru/design/388516-skolko-zarabatyvayut-barking-store-ovd-info-i-motuznaya-na-dizayne-i-prodazhe-svoego-mercha']

In [ ]:
api_id = input('Enter api_id: ')

In [ ]:
api_hash = input('Enter api_hash: ')

In [ ]:
phone = input('Enter phone: ')

In [42]:
!pip install python-telegram

     |████████████████████████████████| 14.9 MB 8.8 MB/s 
  Created wheel for python-telegram: filename=python_telegram-0.15.0-py3-none-any.whl size=15043097 sha256=1cfc9610dc6cf08d03d12461c5bb47c4ad00749bd1b9d796a991e7e26a9dbb4f
  Stored in directory: /root/.cache/pip/wheels/3b/2b/63/98ab0867fb5a63539b3b2426c246bf93adc87511d84b501603
Successfully built python-telegram


In [ ]:
from telegram.client import Telegram


#this requires building tdlib https://github.com/tdlib/td#building , so can't run it here
#https://github.com/alexander-akhmetov/python-telegram/blob/master/examples/send_message.py
def send_msg(api_id, api_hash, phone, chat_id, text):
    tg = Telegram(
        api_id=api_id,
        api_hash=api_hash,
        phone=phone,
        database_encryption_key='tmpval',
    )
    tg.login()
    result = tg.get_chats()
    result.wait()

    if result.error:
        print(f'get chats error: {result.error_info}')
    else:
        print(f'chats: {result.update}')

    result = tg.send_message(
        chat_id=chat_id,
        text=text,
    )

    result.wait()
    if result.error:
        print(f'send message error: {result.error_info}')
    else:
        print(f'message has been sent: {result.update}')

    tg.stop()